# Validation

In [ ]:
! pip install einops
! pip install imutils
! pip install opencv-python
! pip install matplotlib
! pip install scikit-image
! pip install natsort
! pip install numpy

In [1]:
import os
from collections import namedtuple
import cv2
import csv
import kornia as K
import kornia.feature as KF
from kornia.feature.loftr import LoFTR
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import glob
import random
import imutils
from tqdm.notebook import tqdm
from PIL import Image
import matplotlib.cm as cm

from kornia_moons.feature import *

# from src.loftr import LoFTR, default_cfg
from src.utils.plotting import make_matching_figure

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil

# =============================

import torch
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from runpy import run_path
from skimage import img_as_ubyte

from natsort import natsorted
from glob import glob
import cv2
from tqdm import tqdm
import argparse
import numpy as np
from numpy.lib.arraypad import pad

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
matcher = KF.LoFTR(pretrained=None)
matcher.load_state_dict(torch.load("weights/outdoor_ds.ckpt")['state_dict'])
matcher = matcher.to(device).eval()

In [3]:
LONGEST_EDGE = 1280
def resize_keep_ratio(img, longest_size=LONGEST_EDGE):
    height, width = img.shape[:2]
    if np.maximum(height, width) <= longest_size: # no need to resize
        return img
    
    if height >= width:
        resized_img = imutils.resize(img, height=longest_size)
    else:
        resized_img = imutils.resize(img, width=longest_size)
    return resized_img

def load_torch_image(fname):
    img = cv2.imread(fname)
    scale = 840 / max(img.shape[0], img.shape[1]) 
    w = int(img.shape[1] * scale)
    h = int(img.shape[0] * scale)
    img = cv2.resize(img, (w, h))

    # img = resize_keep_ratio(img)
    # img = cv2.resize(img, (img.shape[1]//8*8, img.shape[0]//8*8))  # input size should be divisible by 8

    img = K.image_to_tensor(img, False).float() /255.
    img = K.color.bgr_to_rgb(img)
    return img

In [4]:
def match(img_path0, img_path1, matcher, device=device):
    img0 = load_torch_image(img_path0)
    img1 = load_torch_image(img_path1)
        
    input_dict = {"image0": K.color.rgb_to_grayscale(img0).to(device), 
                  "image1": K.color.rgb_to_grayscale(img1).to(device)}
    
    with torch.no_grad():
        correspondences = matcher(input_dict)
        
    mkpts0 = correspondences['keypoints0'].cpu().numpy()
    mkpts1 = correspondences['keypoints1'].cpu().numpy()
        
    return mkpts0, mkpts1
        
def get_F_matrix(mkpts0, mkpts1):

    # Make sure we do not trigger an exception here.
    if len(mkpts0) > 8:
        F, inliers = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.5, 0.999, 100000)

        assert F.shape == (3, 3), 'Malformed F?'
    else:
        F = np.zeros((3, 3))

    return F

In [5]:
src = '../image-matching-challenge-2022/train'

scaling_dict = {}
with open(f'{src}/scaling_factors.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        # Skip header.
        if i == 0:
            continue
        scaling_dict[row[0]] = float(row[1])

print(f'Scaling factors: {scaling_dict}')

Scaling factors: {'british_museum': 2.517, 'brandenburg_gate': 7.38, 'buckingham_palace': 18.75, 'colosseum_exterior': 36.99, 'grand_place_brussels': 10.26, 'lincoln_memorial_statue': 1.85, 'notre_dame_front_facade': 1.36, 'pantheon_exterior': 5.41, 'piazza_san_marco': 7.92, 'sacre_coeur': 20.27, 'sagrada_familia': 4.2, 'st_pauls_cathedral': 7.01, 'st_peters_square': 21.48, 'taj_mahal': 20.76, 'temple_nara_japan': 7.79, 'trevi_fountain': 3.67}


In [1]:
def restoration(img_pth0, img_pth1):
    # task = 'Real_Denoising'
    task = 'Single_Image_Defocus_Deblurring'
    # task = 'Motion_Deblurring'
    # task = 'Deraining'

    shutil.rmtree('demo')

    input_dir = 'demo/sample_images/'+task+'/degraded'
    os.makedirs(input_dir, exist_ok=True)
    
    shutil.copy(img_pth0, input_dir)
    shutil.copy(img_pth1, input_dir)

    def get_weights_and_parameters(task, parameters):
        if task == 'Motion_Deblurring':
            weights = os.path.join('Motion_Deblurring', 'pretrained_models', 'motion_deblurring.pth')
        elif task == 'Single_Image_Defocus_Deblurring':
            weights = os.path.join('Defocus_Deblurring', 'pretrained_models', 'single_image_defocus_deblurring.pth')
        elif task == 'Deraining':
            weights = os.path.join('Deraining', 'pretrained_models', 'deraining.pth')
        elif task == 'Real_Denoising':
            weights = os.path.join('Denoising', 'pretrained_models', 'real_denoising.pth')
            parameters['LayerNorm_type'] =  'BiasFree'
        return weights, parameters

    # Get model weights and parameters
    parameters = {'inp_channels':3, 'out_channels':3, 'dim':48, 'num_blocks':[4,6,6,8], 'num_refinement_blocks':4, 'heads':[1,2,4,8], 'ffn_expansion_factor':2.66, 'bias':False, 'LayerNorm_type':'WithBias', 'dual_pixel_task':False}
    weights, parameters = get_weights_and_parameters(task, parameters)

    load_arch = run_path(os.path.join('basicsr', 'models', 'archs', 'restormer_arch.py'))
    model = load_arch['Restormer'](**parameters)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    device = torch.device(device)
    # print(device)
    checkpoint = torch.load(weights)
    # print(checkpoint)
    model.load_state_dict(checkpoint['params'])
    model = model.to(device).eval()

    input_dir = 'demo/sample_images/'+task+'/degraded'
    out_dir = 'demo/sample_images/'+task+'/restored'
    os.makedirs(out_dir, exist_ok=True)
    extensions = ['jpg', 'JPG', 'png', 'PNG', 'jpeg', 'JPEG', 'bmp', 'BMP']
    files = natsorted(glob(os.path.join(input_dir, '*')))

    img_multiple_of = 8

    print(f"\n ==> Running {task} with weights {weights}\n ")
    with torch.no_grad():
        for filepath in tqdm(files):
            img = cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)
            input_ = torch.from_numpy(img).float().div(255.).permute(2, 0, 1).unsqueeze(0)

            # Pad the input if not_multiple_of 8
            h, w = input_.shape[2], input_.shape[3]
            H, W = ((h + img_multiple_of) // img_multiple_of) * img_multiple_of, ((w + img_multiple_of) // img_multiple_of) * img_multiple_of
            padh = H - h if h % img_multiple_of != 0 else 0
            padw = W - w if w % img_multiple_of != 0 else 0

            # Convert input_ to NumPy array
            input_ = input_.squeeze(0).permute(1, 2, 0).cpu().numpy()
            input_ = np.pad(input_, ((0, padh), (0, padw), (0, 0)), mode='reflect')
            input_ = torch.from_numpy(input_).permute(2, 0, 1).unsqueeze(0)

            restored = model(input_)
            restored = torch.clamp(restored, 0, 1)

            # Unpad the output
            restored = restored[:, :, :h, :w]

            restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
            restored = img_as_ubyte(restored[0])

            filename = os.path.split(filepath)[-1]
            cv2.imwrite(os.path.join(out_dir, filename), cv2.cvtColor(restored, cv2.COLOR_RGB2BGR))    
    
    new_img_pth0 = out_dir + '/' + os.listdir(out_dir)[0]
    new_img_pth1 = out_dir + '/' + os.listdir(out_dir)[1]
    print(os.listdir(out_dir)[0] + ' RESTORED')
    print(os.listdir(out_dir)[1] + ' RESTORED')
    return new_img_pth0, new_img_pth1

In [2]:
def restoration(img_pth0, img_pth1):
    task = 'Single_Image_Defocus_Deblurring'  # Specify the restoration task

    shutil.rmtree('demo')
    input_dir = f'demo/sample_images/{task}/degraded'
    os.makedirs(input_dir, exist_ok=True)
    
    shutil.copy(img_pth0, input_dir)
    shutil.copy(img_pth1, input_dir)

    def get_weights_and_parameters(task):
        weights = ''
        parameters = {'inp_channels':3, 'out_channels':3, 'dim':48, 'num_blocks':[4,6,6,8], 'num_refinement_blocks':4, 'heads':[1,2,4,8], 'ffn_expansion_factor':2.66, 'bias':False, 'LayerNorm_type':'WithBias', 'dual_pixel_task':False}

        if task == 'Motion_Deblurring':
            weights = os.path.join('Motion_Deblurring', 'pretrained_models', 'motion_deblurring.pth')
        elif task == 'Single_Image_Defocus_Deblurring':
            weights = os.path.join('Defocus_Deblurring', 'pretrained_models', 'single_image_defocus_deblurring.pth')
        elif task == 'Deraining':
            weights = os.path.join('Deraining', 'pretrained_models', 'deraining.pth')
        elif task == 'Real_Denoising':
            weights = os.path.join('Denoising', 'pretrained_models', 'real_denoising.pth')
            parameters['LayerNorm_type'] = 'BiasFree'

        return weights, parameters

    weights, parameters = get_weights_and_parameters(task)

    load_arch = run_path(os.path.join('basicsr', 'models', 'archs', 'restormer_arch.py'))
    model = load_arch['Restormer'](**parameters)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    checkpoint = torch.load(weights, map_location=device)
    model.load_state_dict(checkpoint['params'])
    model = model.to(device).eval()

    out_dir = f'demo/sample_images/{task}/restored'
    os.makedirs(out_dir, exist_ok=True)
    
    img_multiple_of = 8

    print(f"\n==> Running {task} with weights {weights}\n")
    with torch.no_grad():
        files = natsorted(glob(os.path.join(input_dir, '*')))
        for filepath in tqdm(files):
            img = cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)
            input_ = torch.from_numpy(img).float().div(255.).permute(2, 0, 1).unsqueeze(0)

            # Pad the input if not_multiple_of 8
            h, w = input_.shape[2], input_.shape[3]
            H, W = ((h + img_multiple_of) // img_multiple_of) * img_multiple_of, ((w + img_multiple_of) // img_multiple_of) * img_multiple_of
            padh = H - h if h % img_multiple_of != 0 else 0
            padw = W - w if w % img_multiple_of != 0 else 0

            # Convert input_ to NumPy array
            input_ = input_.squeeze(0).permute(1, 2, 0).cpu().numpy()
            input_ = np.pad(input_, ((0, padh), (0, padw), (0, 0)), mode='reflect')
            input_ = torch.from_numpy(input_).permute(2, 0, 1).unsqueeze(0).to(device)

            restored = model(input_)
            restored = torch.clamp(restored, 0, 1)

            # Unpad the output
            restored = restored[:, :, :h, :w]

            restored = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
            restored = img_as_ubyte(restored[0])

            filename = os.path.split(filepath)[-1]
            cv2.imwrite(os.path.join(out_dir, filename), cv2.cvtColor(restored, cv2.COLOR_RGB2BGR))    
    
    new_img_pth0 = os.path.join(out_dir, os.listdir(out_dir)[0])
    new_img_pth1 = os.path.join(out_dir, os.listdir(out_dir)[1])
    print(os.listdir(out_dir)[0] + ' RESTORED')
    print(os.listdir(out_dir)[1] + ' RESTORED')
    return new_img_pth0, new_img_pth1

In [3]:
from imc_metric import EvaluateSubmission, ReadCovisibilityData, FlattenMatrix

ImportError: cannot import name 'src' from '__main__' (unknown location)

In [9]:
max_num_pairs = 3

sample_ids = []
fund_matrices = []

count = 0
scalingLen = len(scaling_dict.keys())

for scene in scaling_dict.keys():
    count += 1
    print('Folder ' + str(count) + '/' + str(scalingLen))

    covisibility_dict, F_gt_dict = ReadCovisibilityData(f'{src}/{scene}/pair_covisibility.csv')

    # Let's remove pairs with a covisibility below 0.1. Note that the keys are roughly sorted by difficulty, in increasing order, so let's also shuffle before subsampling.
    # Neither matters for the purposes of this exercise, but let's prevent mistakes down the line.
    pairs = list([key for key, covis in covisibility_dict.items() if covis >= 0.1])
    random.shuffle(pairs)
    n = len(pairs)
    pairs = pairs[:max_num_pairs]
    # print(pairs)
    print(f'Loading covisibility data for "{scene}"... kept {len(pairs)} out of {n} covisible pairs')
    
    for pair in tqdm(pairs):
        image_0_id, image_1_id = pair.split('-')
        img_path0 = f'{src}/{scene}/images/{image_0_id}.jpg'
        img_path1 = f'{src}/{scene}/images/{image_1_id}.jpg'

        img_path0, img_path1 = restoration(img_path0, img_path1)

        mkpts0, mkpts1 = match(img_path0, img_path1, matcher)
        F = get_F_matrix(mkpts0, mkpts1)
        F_str = FlattenMatrix(F)

        sample_ids.append(f'phototourism;{scene};{pair}')
        fund_matrices.append(F_str)
        # break
    # if count == 3:
    #     break
    # break

Folder 1/16
Loading covisibility data for "british_museum"... kept 5 out of 14793 covisible pairs


  0%|          | 0/5 [00:00<?, ?it/s]


==> Running Single_Image_Defocus_Deblurring with weights Defocus_Deblurring\pretrained_models\single_image_defocus_deblurring.pth



In [ ]:
fund_matrices

['-1.22018306e-07 -6.62613654e-06 3.99141598e-03 5.95987564e-06 6.07795062e-07 1.70037789e-03 -3.63871381e-03 -1.96591492e-03 -6.72505285e-02',
 '-2.41235234e-08 -1.91292170e-05 9.04115090e-03 2.05223475e-05 1.42363615e-06 -5.60514708e-03 -7.00121359e-03 3.50388819e-03 2.77994731e-02',
 '-6.18233235e-08 2.42036373e-05 -1.50568824e-02 -2.44320500e-05 -7.04826017e-08 8.90095838e-03 1.25910132e-02 -1.17638305e-02 2.76461161e+00',
 '-3.35638462e-07 -2.04919190e-06 3.02819507e-03 -3.05961487e-06 8.83777143e-07 1.05327479e-02 1.54757938e-03 -5.15400244e-03 -1.22016902e+00',
 '-6.62005729e-08 3.40665384e-06 -1.52117259e-03 2.54410587e-06 3.65849484e-07 -8.32188128e-03 -1.38267342e-03 6.18289724e-03 1.27656738e+00',
 '1.25325427e-06 3.39444482e-05 -2.22396330e-02 -3.48926299e-05 1.72261748e-06 1.22335080e-02 6.56876200e-03 -1.26199964e-02 5.57929896e+00',
 '-7.14273559e-09 -2.44063252e-06 4.84979726e-04 2.76639098e-06 -4.72855914e-06 -5.38587294e-03 -4.61227358e-04 6.68347797e-03 -2.34207011e-

In [ ]:
df = pd.DataFrame({'sample_id':sample_ids, 'fundamental_matrix':fund_matrices})
df.to_csv('train_pred_nores.csv', index=False)

In [86]:
thresholds_q = np.linspace(1, 10, 10)
thresholds_t = np.geomspace(0.2, 5, 10)

print('--- Evaluate prediction ---')
maa, maa_per_scene, errors_dict_q, errors_dict_t = EvaluateSubmission('train_pred_nores.csv', scaling_dict, thresholds_q, thresholds_t)
for scene, cur_maa in maa_per_scene.items():
    print(f'Scene "{scene}" ({len(errors_dict_q[scene])} pairs), mAA={cur_maa:.05f}')
print()
print(f'Full dataset: mAA={maa:.05f}')
print()

--- Evaluate prediction ---


100%|██████████| 9/9 [00:00<00:00, 1124.75it/s]

Scene "british_museum" (3 pairs), mAA=0.06667
Scene "brandenburg_gate" (3 pairs), mAA=0.00000
Scene "buckingham_palace" (3 pairs), mAA=0.00000

Full dataset: mAA=0.02222

